# Extracting Features from Skeletons

The feature extraction pipeline that generated the dataframe we're working with extracts a number of features from each skeleton.
This notebook will cover some of these features and introduce you to a design pattern for working with your own ideas for features.

In [1]:
import numpy as np
import pandas as pd
from meshparty import meshwork
import matplotlib.pyplot as plt

In [2]:
# Load an example cell from the data we already have

filename = 'data/exampleNets/864691136452201983.h5'
nrn = meshwork.load_meshwork(filename)

## Splitting the problem into pieces

Let's imagine a couple of features. Rather than make one giant function that does all the features we can imagine, let's take an approach where each feature is one function. This will make things more modular and, we will see, make life more simple in the future.

The first feature is the total path length. This one is fairly simple, because there's already a function to compute the path length of the cell. However, we do have to do some work — for example, this is going to just be a dendritic feature, so we need to mask out the axon while doing these computations. We want a function that takes a meshwork object as its input and returns the path length.

In [3]:
def path_length_dendrite(nrn):
    # The mask_context function sets the mask and then resets it after the code inside is run.
    with nrn.mask_context(
        ~nrn.anno['is_axon'].mesh_mask  # the ~ is a logical not, so this is taking everything that is *not*
    ) as nmc:
        return nmc.path_length() / 1_000 # Convert to microns from nanometers by dividing by 10^3

In [4]:
path_length_dendrite(nrn)

3117.29

Now let's do something with synapses. For example, let's compute the total number and median size of input synapses on the dendrite:

In [5]:
def input_synapse_count(nrn):
    return len(
        nrn.anno.post_syn.filter_query(
             ~nrn.anno['is_axon'].mesh_mask
        ).df
    )
    
def input_synapse_median_size(nrn):
    size_values = nrn.anno.post_syn.filter_query(
        ~nrn.anno['is_axon'].mesh_mask
    ).df['size'].values
    return np.median(size_values)

In [6]:
input_synapse_count(nrn)

4333

In [7]:
input_synapse_median_size(nrn)

4188.0

### Exercise

How about just getting the number of somatic synapses?

*Reminder: To get a mask that is for the soma alone, you can use* `nrn.root_region.to_mesh_mask`

In [8]:
def soma_synapse_count(nrn):
    pass
    
def soma_synapse_median_size(nrn):
    pass

### Array output instead of values

Now let's add one final nuance, and rather than return a value we'll return an array. The idea here is that we can do some computation based on this array later using information that comes from all the cells collectively — for example, PCA components of some binned property.

Let's start by making a function that counts how many synapses of a cell is in each of a depth bin, using the same bins for all cells. In this case, we will split each bin into 20 microns starting at the pia surface, using `standard_transform` to rotate, scale, and translate the synapse coordinates.

In [9]:
from standard_transform import minnie_ds

bins = np.linspace(0,900,46)

def synapse_depth_binned(nrn):
    # This uses standard transform to get the depth projection of the synapse location.
    with nrn.mask_context(~nrn.anno['is_axon'].mesh_mask) as nmc:
        counts, _bins = np.histogram(
            minnie_ds.transform_vx.apply_dataframe(
                    'ctr_pt_position',
                    nmc.anno.post_syn.df,
                    projection='y',
                ),
            bins=bins,
        )
        return counts

## Assembling the features together

Now let's assemble all of these functions into one collection in order to generate a collection of features (including the two examples you might have written):

The gist is that we're going to make a function that takes in a meshwork object (`nrn`) and returns a dictionary where each key is a feature name and each value is the feature value:

In [10]:
def get_features(nrn):
    return {
        'path_length': path_length_dendrite(nrn),
        'input_syn_count': input_synapse_count(nrn),
        'input_syn_median_size': input_synapse_median_size(nrn),
        # 'soma_syn_count': soma_synapse_count(nrn),
        # 'soma_syn_median_size': soma_synapse_median_size(nrn),   # Uncomment these two lines if you put in a solution to the functions above
        'synapse_depth_binned': synapse_depth_binned(nrn),
    }

The nice thing about this is that it's very modular. If you want to add a feature, write a function and add a line here. Remove a feature? Delete a line. Edit a feature? Change that one function. Easy to read and easy to edit or debug.

Moreover, the arrangement as a dict makes it easy to run across several neurons and put into a pandas dataframe.

In [16]:
# These are just the root ids with skeletons already in the example data folder
root_ids = [864691134965653279, 864691135208968313, 864691135759684174, 864691135866737541, 864691136452201983, 864691136953057759]

In [12]:
feats = []
for root_id in root_ids:
    filename = f'data/exampleNets/{root_id}.h5'
    nrn = meshwork.load_meshwork(filename)
    feats.append(
        get_features(nrn)
    )

In [13]:
feature_df = pd.DataFrame(
    feats,
    index=root_ids,
)

feature_df

,path_length,input_syn_count,input_syn_median_size,synapse_depth_binned
864691134965653279,2153.34750,1011,5016.0,"[8, 48, 31, 19, 18, 22, 6, 2, 10, 16, 41, 240,..."
864691135208968313,5420.38800,5677,4684.0,"[196, 629, 556, 1092, 940, 932, 783, 409, 109,..."
864691135759684174,2572.20175,1574,4714.0,"[0, 3, 14, 9, 11, 8, 6, 11, 10, 12, 10, 11, 16..."
864691135866737541,7450.01100,10017,4580.0,"[89, 219, 234, 157, 162, 90, 97, 51, 60, 111, ..."
864691136452201983,3117.29000,4333,4188.0,"[0, 0, 0, 0, 17, 14, 19, 20, 24, 28, 38, 36, 8..."
864691136953057759,5395.45650,6342,4540.0,"[13, 239, 286, 278, 216, 168, 310, 673, 557, 8..."


## A few extra notes:

You can see the functions used in the `skel_features` directory. They are a bit more complicated than what we did here, but only to make them a bit more generic. For example, in order to use potentially different compartments other than just axon/dendrite. In addition, because this was run on many more cells while proofreading was going on, it made sense to be able to store the results of feature extraction as a file (in this case a JSON file), so that if a cell had already had features extracted the values could be looked up instead of re-computed.

These were intended as very simple features, and there's a lot more to discover or measure if you want! Be creative!